# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests. 

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content. 

In [1]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))


Let's define a prompt outlining a dialogue chat bot.

In [2]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [3]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

Initialize your context

In [4]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [5]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Sure! What genre or type of books are you interested in?


Update the history with the output

In [6]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure! What genre or type of books are you interested in?



Keep Chatting!

In [7]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [8]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: I would recommend "The Histories" by Herodotus. It is considered one of the earliest works of history and provides a detailed account of ancient Greece, including the Persian Wars. Another great book is "The Republic" by Plato, which explores various philosophical ideas and includes discussions on justice, politics, and the ideal society.


In [9]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Histories" by Herodotus is a collection of nine books that provide a comprehensive account of various events and cultures of the ancient world, with a particular focus on the Persian Wars. Herodotus explores the causes and consequences of the wars, delving into the political, social, and cultural aspects of the time. He also includes fascinating anecdotes and stories about the people and places he encounters during his travels.

"The Republic" by Plato is a philosophical dialogue that explores the nature of justice, the ideal society, and the role of the philosopher in governance. The book is set as a conversation between Socrates and several other characters, and it covers a wide range of topics, including the nature of reality, the forms of government, and the education of citizens. It is considered one of the most influential works in Western philosophy.


In [10]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read "The Histories" by Herodotus, you will learn about various aspects of Greek history during the Persian Wars. Herodotus provides detailed accounts of the events leading up to the wars, the battles themselves, and their aftermath. He also explores the political and cultural dynamics of ancient Greece, including the city-states, their alliances, and their interactions with other civilizations.

Through Herodotus' narrative, you will gain insights into the political systems, military strategies, and cultural practices of ancient Greece. You will also learn about the interactions between the Greeks and the Persians, as well as the cultural and religious differences between the two civilizations.

Overall, "The Histories" offers a comprehensive and engaging exploration of Greek history during a significant period, providing valuable insights into the ancient world.


In [11]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are some more books you could consider reading about Greek history:

1. "The Peloponnesian War" by Thucydides - This book provides a detailed account of the Peloponnesian War between Athens and Sparta, offering insights into the political and military strategies of the time.

2. "The Greek Way" by Edith Hamilton - This book explores the cultural and intellectual achievements of ancient Greece, discussing topics such as art, literature, philosophy, and politics.

3. "The Greek World 479-323 BC" by Simon Hornblower - This book provides a comprehensive overview of Greek history during the Classical period, covering topics such as the rise of Macedon, the conquests of Alexander the Great, and the Hellenistic world.

4. "The Rise and Fall of Athens: Nine Greek Lives" by Plutarch - This book offers biographies of nine prominent figures from ancient Athens, including Pericles, Themistocles, and Alcibi

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [12]:
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Sure! What genre or type of books are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: I would recommend "The Histories" by Herodotus. It is considered one of the earliest works of history and provides a detailed account of ancient Greece, including the Persian Wars. Another great book is "The Republic" by Plato, which explores various philosophical ideas and includes discussions on justice, politics, and the ideal society.

User: that sounds interesting, what is it about?
ChatBot: "The Histories" by Herodotus is a collection of nine books that provide a comprehensive account of various events and cultures of the ancient world, with a particular focus on the Persian Wars. Herodotus explores the causes and consequences of the wars, delving into the political, social, and cultural aspects of the time. He also includes fascinating anecdotes and stories about 